<font color="green">*To start working on this notebook, or any other notebook that we will use in the Moringa Data Science Course, we will need to save our own copy of it. We can do this by clicking File > Save a Copy in Drive. We will then be able to make edits to our own copy of this notebook.*</font>

# SQL Programming - Subqueries using Select Statements

## 1.0 Connecting to our Database

In [1]:
# We will first load the sql extension into our environment
%load_ext sql

# Then connect to our in memory sqlite database
# NB: This database will cease to exist as soon as the database connection is closed. 
%sql sqlite://

'Connected: @None'

## 1.1 Loading Data from CSV Files

Datasets used in this notebook:


1.   **World Countries**

> ([Dataset Download Link](https://drive.google.com/a/moringaschool.com/file/d/1XlabneVWhvbmpRUi_Wix2FR5uYAAp-ZY/view?usp=sharing))

2.   **World Cities** 

> ([Dataset Download Link](https://drive.google.com/a/moringaschool.com/file/d/1R5952m-8hH91uiwDK8dcL9sT2D_PEcuw/view?usp=sharing))






In [2]:
# Importing Pandas
import pandas as pd

In [3]:
# Loading our Countries dataset
# 

with open("world_countries.csv", "r") as w:
  world_countries = pd.read_csv(w, index_col=0, encoding='utf-8')

%sql DROP TABLE if EXISTS world_countries;
%sql PERSIST world_countries;

 * sqlite://
Done.
 * sqlite://


'Persisted world_countries'

In [4]:
# Loading our cities dataset 
# 
with open("world_cities.csv", "r") as w:
  world_cities = pd.read_csv(w, index_col=0, encoding='utf-8')

%sql DROP TABLE if EXISTS world_cities;
%sql PERSIST world_cities;

 * sqlite://
Done.
 * sqlite://


'Persisted world_cities'

## 1.2 Subqueries using the SELECT statement

A subquery is a query within another query. The outer query is called as main query and inner query is called as subquery. The subquery must be enclosed with parenthesis and generally executes first, and its output is used to complete the query condition for the main or outer query. We will go through examples of how subqueries can be used to answers different types of questions then later, we will use our learnings to work on the challenges.

In [5]:
%%sql
SELECT * FROM world_cities LIMIT 10

 * sqlite://
Done.


city,country,latitude,longitude,temperature
Aalborg,Denmark,57.03,9.92,7.52
Aberdeen,United Kingdom,57.17,-2.08,8.1
Abisko,Sweden,63.35,18.83,0.2
Adana,Turkey,36.99,35.32,18.67
Albacete,Spain,39.0,-1.87,12.62
Algeciras,Spain,36.13,-5.47,17.38
Amiens,France,49.9,2.3,10.17
Amsterdam,Netherlands,52.35,4.92,8.93
Ancona,Italy,43.6,13.5,13.52
Andorra,Andorra,42.5,1.52,9.6


In [6]:
%%sql
SELECT * FROM world_countries LIMIT 10

 * sqlite://
Done.


country,population,EU,coastline
Albania,2.9,no,yes
Andorra,0.07,no,no
Austria,8.57,yes,no
Belarus,9.48,no,no
Belgium,11.37,yes,yes
Bosnia and Herzegovina,3.8,no,yes
Bulgaria,7.1,yes,yes
Croatia,4.23,yes,yes
Cyprus,1.18,yes,yes
Czech Republic,10.55,yes,no


In [8]:
# Example 1
# Subqueries can occur in the select list of the containing query
# as shown below;
# Let's find all countries in the Countries table with no city in the Cities table
# 
%%sql
SELECT country
FROM world_countries
WHERE NOT EXISTS (SELECT * FROM world_cities
                  WHERE world_cities.country = world_countries.country) 


 * sqlite://
Done.


country
Cyprus
Iceland
Kosovo
Liechtenstein
Luxembourg


In [9]:
# Example 2
# Find the European countries with cities experiencing temperature > 20?
# 
%%sql
SELECT country
FROM world_countries
WHERE EU = 'yes'
AND EXISTS (SELECT * FROM world_cities
            WHERE world_cities.country = world_countries.country
            AND temperature > 15)

 * sqlite://
Done.


country
Greece
Italy
Portugal
Spain


In [ ]:
# Example 3: Method 1
# Find the westernmost city, returning the city and longitude
# 
%%sql
SELECT city, longitude
FROM world_cities C1
WHERE NOT EXISTS (SELECT * FROM world_cities C2
                  WHERE C2.longitude < C1.longitude)

 * sqlite://
Done.


city,longitude
Lisbon,-9.14


In [ ]:
# Example 3: Method 2
# Find the westernmost city, returning the city and longitude
# 
%%sql
SELECT " city", longitude
FROM world_cities
WHERE longitude = (SELECT min(longitude) FROM world_cities)

 * sqlite://
Done.


""" city""",longitude
city,-9.14


In [ ]:
#from pandas.core.dtypes.dtypes import CategoricalDtype
# Example 4:
# How many cities are in the EU?
# 
%%sql
SELECT count(country)
FROM world_cities
WHERE country IN (SELECT country FROM world_countries WHERE EU = 'yes')

 * sqlite://
Done.


count(country)
150


### <font color="green">1.2 Challenges</font>

In [ ]:
# Challenge 1
# Find number of countries that have a city with latitude > 1000 (start with country list)
# 
%%sql
SELECT COUNT(country) FROM world_countries
WHERE country IN ( SELECT city FROM world_cities WHERE latitude > 1000)

 * sqlite://
Done.


COUNT(country)
0


In [29]:
# Challenge 2
# Find three western most cities returning both the cities and their longitude
#
%%sql
SELECT city,longitude FROM world_cities
WHERE longitude IN (SELECT longitude FROM world_cities wHERE longitude < 0) LIMIT 3

 * sqlite://
Done.


city,longitude
Aberdeen,-2.08
Albacete,-1.87
Algeciras,-5.47


In [19]:
# Challenge 3
# Find cities with temperature is more than 50% higher than the average; 
# We will return the city, country, and temperature, ordered by descending temperature
# Hint: avg(temperature) * 1.5
# 
%%sql 
SELECT city,country,temperature FROM world_cities
WHERE temperature > (SELECT AVG(temperature) * 1.5 FROM world_cities)

 * sqlite://
Done.


city,country,temperature
Adana,Turkey,18.67
Algeciras,Spain,17.38
Athens,Greece,17.41
Badajoz,Spain,15.61
Barcelona,Spain,15.78
Bari,Italy,15.15
Cartagena,Spain,17.32
Catania,Italy,15.04
Cosenza,Italy,16.6
Denizli,Turkey,15.02


In [25]:
# Challenge 4
# How many cities are not in the EU and without a coastline?
# Hint: "not in"
# 
%%sql 
SELECT country FROM world_countries LIMIT 3
WHERE country NOT IN (EU="yes")


 * sqlite://
(sqlite3.OperationalError) near "WHERE": syntax error
[SQL: SELECT country FROM world_countries LIMIT 3
WHERE country NOT IN (EU="yes")]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


In [27]:
# Challenge 5
# Find the number of countries with no coastline and a city with longitude < 10
# 
%%sql
SELECT country FROM world_countries
WHERE coastline = "no" IN (SELECT city FROM world_cities WHERE )


 * sqlite://
Done.


country
Andorra
Austria
Belarus
Czech Republic
Hungary
Kosovo
Liechtenstein
Luxembourg
Macedonia
Moldova


In [ ]:
# Challenge 6
# Find countries in Countries table with no city in Cities table
# 
%%sql
SELECT country
FROM world_countries
WHERE NOT EXISTS (SELECT city FROM world_cities )

 * sqlite://
Done.


country


In [39]:
# Challenge 7
# Find all cities in a country whose population is < 2, returning the city and country.
# 
%%sql
SELECT city, country FROM world_cities
WHERE country IN (SELECT country FROM world_countries WHERE population < 2)

 * sqlite://
Done.


city,country
Andorra,Andorra
Daugavpils,Latvia
Podgorica,Montenegro
Riga,Latvia
Tallinn,Estonia
Tartu,Estonia


In [41]:
# Challenge 8
# Find all countries with no city having a temperature > 6
# 
%%sql
SELECT country FROM world_countries
WHERE city NOT IN (SELECT city FROM world_countries WHERE temperature > 6)


 * sqlite://
(sqlite3.OperationalError) no such column: city
[SQL: SELECT country FROM world_countries
WHERE city NOT IN (SELECT city IN world_countries WHERE temperature > 6)]
(Background on this error at: http://sqlalche.me/e/14/e3q8)
